In [ ]:

time_misic=np.ones((len(n_feature_set),len(trees_to_test),n_rep))
time_proj=np.ones((len(n_feature_set),len(trees_to_test),n_rep))
time_bigm=np.ones((len(n_feature_set),len(trees_to_test),n_rep))
time_misic_cuts=np.ones((len(n_feature_set),len(trees_to_test),n_rep))

for j,n_feature_trunc in enumerate(n_feature_set):
    for i,tree_size in enumerate(trees_to_test):
        for k in range(n_rep):
            x,y,feature_upper_bounds,feature_lower_bounds,n_data,n_feature=choose_dataset(dataset,n_data,n_feature_trunc)
            
            if dataset!='abs_synth_noisy':
                x=x.iloc[:, 0:n_feature_trunc]
                feature_upper_bounds[0:n_feature_trunc]
                feature_lower_bounds[0:n_feature_trunc]
            n_feature=n_feature_trunc
            
            print("fitting tree")
            clf = RandomForestRegressor(n_estimators=tree_size,max_depth=20)
            clf.fit(x, y)
            print("tree fit \n")

            num_leaves=[]
            depth=[]
            for tree_index,tree_in_forest in enumerate(clf.estimators_):

                tree=tree_in_forest.tree_
                #pprint(getmembers(tree))
                #print()
                num_leaves.append(np.sum(tree.children_left== (-1)))
                depth.append(tree.max_depth)

            total_leaves=np.sum(num_leaves)
            total_depth=np.sum(depth)

            print("total_leaves",total_leaves)
            print("total_depth",total_depth)
            
            # projected
            start_time_proj = timeit.default_timer()
            feature_var,obj,leaf_out, time_proj[j,i,k]=projected_extended_random_forest_IP_unconstrained(clf.estimators_,n_feature,feature_upper_bounds,feature_lower_bounds,x, eps,deviation_bound)
            end_time_proj = timeit.default_timer()
            
            # misic
            start_time_misic = timeit.default_timer()
            obj,leaf, time_misic[j,i,k]=misic_opt(clf.estimators_,n_feature,feature_upper_bounds,feature_lower_bounds,x, eps,make_continuous,deviation_bound,make_relax)
            end_time_misic = timeit.default_timer()
            
            # big M
            start_time_bigm = timeit.default_timer()
            obj,leaf, time_bigm[j,i,k]=tighter_random_forest_IP_unconstrained(clf.estimators_,n_feature,make_relax)
            end_time_bigm = timeit.default_timer()
            
            # elbow
            start_time_misic_cuts = timeit.default_timer()
            obj,leaf, time_misic_cuts[j,i,k]=misic_opt_tighter_with_cuts_2(clf.estimators_,n_feature,feature_upper_bounds,feature_lower_bounds,x, eps,make_continuous,deviation_bound,make_relax)
            end_time_misic_cuts = timeit.default_timer()

    
trees_tested=np.ones((len(n_feature_set),len(trees_to_test),n_rep))
n_feature_tested=np.ones((len(n_feature_set),len(trees_to_test),n_rep))

for j,n_feature in enumerate(n_feature_set):
    for i,tree_size in enumerate(trees_to_test):
        for k in range(n_rep):
            trees_tested[j,i,k]=tree_size
            n_feature_tested[j,i,k]=n_feature

d = {'projected': time_proj.flatten(), 'misic': time_misic.flatten(), 'bigM': time_bigm.flatten(),'cuts': time_misic_cuts.flatten(),'num_trees':trees_tested.flatten(),'n_feature':n_feature_tested.flatten()}
df = pd.DataFrame(data=d)

df.to_csv("output/"+str(dataset)+"_sims_plus_cuts_2_short.csv")

for j,n_feature in enumerate(n_feature_set):

    d = {'projected': time_proj[j,:,:].flatten(), 'misic': time_misic[j,:,:].flatten(), 
         'bigM': time_bigm[j,:,:].flatten(),'elbow': time_misic_cuts[j,:,:].flatten(),'num_trees':trees_tested[j,:,:].flatten()}
    df = pd.DataFrame(data=d)

    means=df.groupby(['num_trees'],as_index=False).mean()
    stds=df.groupby(['num_trees'],as_index=False).std()
    temp=pd.melt(means, id_vars =['num_trees'])
    temp['std_dev']=pd.melt(stds, id_vars =['num_trees']).value

    fig = line(
        data_frame = temp,
        x = 'num_trees',
        y = 'value',
        error_y = 'std_dev',
        error_y_mode = 'band',
        color = 'variable',
        title = '',
        markers = '.',
    )
    yaxis_title="Y Axis Title",
    fig.update_yaxes(type="log")
    fig.update_xaxes(type="log")
    #             fig.update_layout(yaxis_title="Distance to optimal",width=500, height=500,yaxis_range=[0,0.5])
    fig.update_layout(yaxis_title="Solve time (s)",width=500, height=500)
    fig.update_layout(legend={'title_text':''})
    fig.update_layout(legend=dict(orientation = "h", yanchor="top",y=1.15))
    fig.show()
    fig.write_image("plots/"+str(dataset)+"_with_cuts_feat_"+str(n_feature)+".pdf")
    
    